In [19]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, r2_score
from pathlib import Path 


In [26]:
class ScenicDataset(Dataset):
    def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx]['filename'])
        try:
            image = Image.open(img_name).convert('RGB')
        except (IOError, SyntaxError) as e:
            return None  

        if self.transform:
            image = self.transform(image)

        return image  


In [27]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [28]:
from torchvision.models import ResNet50_Weights

class ResNet50FeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNet50FeatureExtractor, self).__init__()
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)  
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  

    def forward(self, x):
        with torch.no_grad():
            features = self.feature_extractor(x)
        return features.squeeze()


In [29]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [30]:
input_dim = 2048  


feature_extractor = ResNet50FeatureExtractor()
feature_extractor.eval()  

model = RegressionModel(input_dim)
model.load_state_dict(torch.load('/home/ubuntu/landscape-aesthetics/models/resnet50_weights.pth', weights_only=True))
model.eval() 


RegressionModel(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [43]:
new_data_path = '/home/ubuntu/landscape-aesthetics/data/external/scenicornot/scenicornot.metadata.csv'
root_dir = '/home/ubuntu/landscape-aesthetics/data/external/scenicornot'

new_data = pd.read_csv(new_data_path)

data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


new_image_dataset = ScenicDataset(data_frame=new_data, root_dir=root_dir, transform=data_transforms)
new_loader = DataLoader(new_image_dataset, batch_size=32, shuffle=False, num_workers=4)

In [48]:
predictions = []
image_filenames = new_data['filename'].tolist()

with torch.no_grad():
    for images in new_loader:
        if images is None:
            continue  

        features = feature_extractor(images)
        features = features.view(features.size(0), -1)

        outputs = model(features)

        predictions.extend(outputs.cpu().numpy())
        

image_filenames = image_filenames[:len(predictions)]

print(f"First 10 predictions: {predictions[:10]}")

new_data['predict_result'] = predictions


df_results.to_csv(new_data_path, index=False)

print("results of prediction have been saved")

First 10 predictions: [array([3.8283322], dtype=float32), array([4.1133823], dtype=float32), array([3.7107525], dtype=float32), array([3.820698], dtype=float32), array([3.3600016], dtype=float32), array([5.5198455], dtype=float32), array([3.829258], dtype=float32), array([5.1671195], dtype=float32), array([5.4208593], dtype=float32), array([5.230237], dtype=float32)]
results of prediction have been saved
